# CFRM 521 - Final Report

#### By: Wooseok (Jeff), Max, Steve, Ilse, Jasmine

----

# Introduction

# Data

In [ ]:
# Load the packages (at the end we can consolidate all the packages we individually used here so that we can create a single environment.yml file for a conda environment at the end)
import pandas as pd 
import numpy as np 
import os
import requests
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from scipy.stats import uniform, loguniform

# Read in the data (this section should be the same for everyone)


# Max Black: SVR

The model for this section will be support vector regression. We chose support vector regression because we are working with options pricing and expect to see complex non-linear relationships. Support vector regression is an excellent candidate for this task, but does have a few drawbacks. Support vector regression scales very slowly with the number of data entries, which means we are limited by our hardware for the purposes of this project. Running on more than around 30,000 data points triggers runtimes that makes optimizing hyperparameters extremely difficult. 

Prior to training our support vector regressor, we will train a naive model that predicts each point to be the mean of our training data. We expect our fully fitted and optimized support vector regressor to outperform this considerably. In order to show this, we will be using mean squared error as our core loss function, but will also examine mean absolute error so we can tell how much outliers are impacting the results. We will also analyze residuals to determine to better understand overall model performance. 

In [ ]:
feature_cols = ['close', 'strike', 'delta', 'gamma',
                'vega', 'theta', 'implied_volatility', 'time_to_expiration']

X_train = train[feature_cols]
y_train = train['mid_price']

X_valid = valid[feature_cols]
y_valid = valid['mid_price']

X_test = test[feature_cols]
y_test = test['mid_price']

We will start by training a scaler on the training data, and applying it to the validation and test sets. 

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

## Naive Model

To determine the efficacy of our support vector regression we will first train a naive model using the mean of our training data. As mentioned before, this model predicts every point to be the mean of the training data. 

In [ ]:
dummy = DummyRegressor(strategy='mean')

Below is a function to plot learning curves for mean absolute error and root mean squared error. It is based on a function introduced in lecture, but adjusted to reduce the number of plotted points. Support vector regression is a very slow process, and plotting a learning curve with a different training set for every entry in the training set, which is what the original function did, is impractical, especially considering the size of our data. For the naive model though, this is not a problem. 

In [ ]:
def plot_learning_curves(model, X_train, y_train, X_val, y_val, n_points=50):
    train_sizes = np.linspace(10, len(X_train), n_points, dtype=int)
    
    train_rmse, val_rmse = [], []
    train_mae, val_mae = [], []

    for m in train_sizes:
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)

        train_rmse.append(np.sqrt(mean_squared_error(y_train[:m], y_train_predict)))
        val_rmse.append(np.sqrt(mean_squared_error(y_val, y_val_predict)))

        train_mae.append(mean_absolute_error(y_train[:m], y_train_predict))
        val_mae.append(mean_absolute_error(y_val, y_val_predict))

    fig, axs = plt.subplots(1, 2, figsize=(10, 7))

    axs[0].plot(train_sizes, train_rmse, "r-+", linewidth=2, label="Train")
    axs[0].plot(train_sizes, val_rmse, "b-", linewidth=3, label="Validation")
    axs[0].set_title("Learning Curve (RMSE)", fontsize=14)
    axs[0].set_xlabel("Training set size", fontsize=12)
    axs[0].set_ylabel("RMSE", fontsize=12)
    axs[0].legend()
    axs[0].grid(True)

    axs[1].plot(train_sizes, train_mae, "r-+", linewidth=2, label="Train")
    axs[1].plot(train_sizes, val_mae, "b-", linewidth=3, label="Validation")
    axs[1].set_title("Learning Curve (MAE)", fontsize=14)
    axs[1].set_xlabel("Training set size", fontsize=12)
    axs[1].set_ylabel("MAE", fontsize=12)
    axs[1].legend()
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
plot_learning_curves(dummy, X_train_scaled, y_train, X_valid_scaled, y_valid, n_points=50)

From these results, we can clearly see that the naive model is underfitting the data. The RMSE and MAE do appear to converge, but to large values of RMSE and MAE. 

Below we plot the actual and predicted values of the naive model along with the MSE and MAE of our naive model on the test set. 

In [ ]:
y_pred_test_dummy = dummy.predict(X_test_scaled)

mse_dummy = mean_squared_error(y_test, y_pred_test_dummy)
mae_dummy = mean_absolute_error(y_test, y_pred_test_dummy)

print(f"\nTest MSE: {mse_dummy:.4f}")
print(f"Test MAE: {mae_dummy:.4f}")

plt.scatter(y_test, y_pred_test_dummy)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual Option Mid-Prices")
plt.ylabel("Predicted Option Mid-Prices")
plt.title("Prediction Vs Actual Option Mid-Prices")
plt.grid(True)
plt.show()

The predicted prices are constant throughout as expected. Clearly this does a poor job of estimating our option prices. We can further see that the test MSE and MAE are extremely high. 

Below we plot the distribution of the model residuals.

In [ ]:
residuals_dummy = y_test - y_pred_test_dummy
plt.hist(residuals_dummy, bins=50, density=True)
plt.title("Residual Distribution for Naive Model")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

We predict the mean, so we would expect the residuals to cluster around 0, which they do. It does appear to be negatively skewed, which

In [ ]:
stats.probplot(residuals_dummy, dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals for Naive Model")
plt.grid(True)
plt.tight_layout()
plt.show()

## Support Vector Regression

We will run a quick pilot study to narrow our search down to one kernel. Ideally would would explore each of them more, but given the complexity of our data structure we are confident that the RBF kernel will be most effective. This is most commonly used when pricing options with SVR, as seen in Wang and Zhang (2010) and Andreou et Alias (2009). We will test each kernel with default hyperparameters to ensure that the RBF kernel is outperforming other kernels, but will not examine this further. 

In [ ]:
kernels = ['linear', 'poly', 'sigmoid', 'rbf',]
results = {}

for kernel in kernels:
    print(f"\nSVR with {kernel} Kernel")

    model = SVR(kernel=kernel, C=1.0, epsilon=0.1)

    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_valid_scaled)

    mse = mean_squared_error(y_valid, y_pred)

    mae = mean_absolute_error(y_valid, y_pred)

    results[kernel] = {'MSE': mse, 'MAE': mae}
    print(f"Validation MSE: {mse:.4f}")
    print(f"Validation MAE: {mae:.4f}")

Based on these results, we can see that our SVR performed best using the RBF kernel, as we expected. The difference is significant by all metrics.

Now that we have analyzed our naive RBF kernel model we will move on to tuning the hyperparameters to see if it significantly improves the accuracy. For hyperparameter tuning we are going to utilize the random search method. We are working with 3 parameters that can take on a continuous range of values. To briefly summarize each of the parameters

- $C$ controls the degree to which margin violations are penalized
- $\varepsilon$ controls the margin of error that we ignore
- $\gamma$ controls how 'local' the decision boundary is

Each of three parameters can take the value of any postive real number. Below we test a range of these values with the RBF kernel. 

In [ ]:
model_rbf = SVR(kernel='rbf')

param_distributions = {
    'C': loguniform(1e-1, 1e3),       
    'gamma': loguniform(1e-4, 1),      
    'epsilon': uniform(0.01, 0.3)    
}

search = RandomizedSearchCV(
    model_rbf,
    param_distributions=param_distributions,
    n_iter=50,             
    scoring='neg_mean_squared_error',
    cv=3,                    
    n_jobs=-1,              
    random_state=42
)

search.fit(X_train_scaled, y_train)

print("Best Parameters Found:")
print(search.best_params_)

print("\nBest CV MSE:", -search.best_score_)

y_pred = search.predict(X_valid_scaled)
mse = mean_squared_error(y_valid, y_pred)
mae = mean_absolute_error(y_valid, y_pred)

print(f"\nValidation MSE: {mse:.4f}")
print(f"Validation MAE: {mae:.4f}")

As we can see the high $C$ value, this model penalizes errors severely. Still it selects a relatively high epsilon of 0.3, so it accepts a small margin of error. The model selects a moderate gamma, not making decisions boundary too local, and allowing the model to generalize slightly more. 

Below we call the function to generate learning curves for our SVR regressor with the RBF kernel and optimized hyperparameters. We can see already from the result above that this model performs well on the validation sets, with MSE of less than 1 and MAE error of less than 0.3. Recall from our naive model that the MSE was over 2000, so this result is encouraging. Note that the validation data shown in the plots below is from the mean across our 3 cross-validation folds in the training data, not the validation dataset that we seperated at the beginning. The calidation predictions are meant to give us some idea about model performance at this stage. 

In [ ]:
best_model_plot = SVR(kernel='rbf', **search.best_params_)

plot_learning_curves(best_model_plot, X_train_scaled, y_train, X_valid_scaled, y_valid)

Now we can determine the results of running this support vector regression on our test data. 

We can see that the errors decrease rapidly as training size increases, with a sharp elbow at aroun 1000 data points. The validation and training sets RMSE and MAE converge to very lower values, with minimal seperation between the two lines. This indicates that our model is neither overfitting or underfitting. 

In [ ]:
best_model = search
y_pred_test = best_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)

print(f"\nTest MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")

The results from our test data show a MSE of less then when, which is a strong result. We saw that our naive data had a MSE of over 2000, so clearly the support vector regressor with RBF kernel is outperformming that. We will now look at how the residuals from the support vector regressor behave. 

In [ ]:
plt.scatter(y_test, y_pred_test, alpha = 0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual Option Mid-Prices")
plt.ylabel("Predicted Option Mid-Prices")
plt.title("Prediction Vs Actual Option Mid-Prices")
plt.grid(True)
plt.show()

From this plot of the predicted vs actual option prices we can see that the every point is close to the actual values. There appears to be some more variance in results closer to zero, but for higher prices the results appear to perform consistently very well. 

In [ ]:
residuals = y_test - y_pred_test
plt.hist(residuals, bins=50, density=True)
plt.title("Residual Distribution")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

From this plot of our residuals, we can see that the vast majority are clustered around zero. This is a good sign, and tells use that our support vector regression likely does not have a positive or negative skew. 

Below we also plot a QQ-plot. Note that there is no assumption of normality of error terms in support vector regression, like there is in linear regression, so we dont actually expect the errors to be normally distributed. We are only doing this to better understand the distribution of the residuals. 

In [ ]:
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q Plot of Residuals for SVR")
plt.grid(True)
plt.tight_layout()
plt.show()

We can see that there are significantly heavier postive and negative tails than you would typically see in a normal distribution. This makes sense, because the model is indifferent to small residuals that fall inside the band created by the hyperparameter $\varepsilon$. This means that residuals can spread out freely near the edge of this bounadary, making the distribution have significantly heavier tails than a normal distribution. This behavior is expected, but is a downside of the model. The width of epsilon could become more significant of a factor for options that have a low prices point. In fact we saw that to some degree when we plotted the actual and predicted values against one another. At lower price points there was a more obvious spread, which suggests that this band was having a noticable impact on the predicted price. We need to be very careful when using it to price extremely low price options, because in those price ranges the band created by epsilon is much more noticable. 

Overall this model is very promising, but has could have issues modeling options in low price ranges. Still it performs very constitently throughout the training, validation and test sets, and generates a low MSE and MAE for the test set, which indicates it is able to price these options effectively. There are also no extreme outliers in our results, which tells us this model performs consistently. 

----

# Wooseok's Model

----


# Steve's Model

----

# Ilse's Model

----

# Jasmine's Model

----

# Conclusions